In [ ]:
# res_type_map = {
#     'A': 0,
#     'C': 4,
#     'D': 3,
#     'E': 6,
#     'F': 13,
#     'G': 7,
#     'H': 8,
#     'I': 9,
#     'K': 11,
#     'L': 10,
#     'M': 12,
#     'N': 2,
#     'P': 14,
#     'Q': 5,
#     'R': 1,
#     'S': 15,
#     'T': 16,
#     'V': 19,
#     'W': 17,
#     'Y': 18,
#     'DA': 20,
#     'DG': 21,
#     'DC': 22,
#     'DT': 23
}

# gamma = []
# for i in range(24):
#     for j in range(i, 24):
#         gamma.append(matrix[i, j])

# # The 21 line should be ALA-DA
# # The 22 line should be ALA-DG
# # The 23 line should be ALA-DC
# # The 24 line should be ALA-DT

In [2]:
# Import necessary libraries
import pandas as pd

# Load the CSV file
file_path = "C2H2.csv"
df = pd.read_csv(file_path)

# Drop rows where 'base' or 'aa' contain NA values
df_cleaned = df.dropna(subset=["base", "aa"])

# Compute the mean coefficient for each (base, aa) pair
df_avg_coef = df_cleaned.groupby(["base", "aa"])["coef"].mean().reset_index()

df_avg_coef


,base,aa,coef
0,A,A,-0.451686
1,A,C,-0.021628
2,A,D,-0.313047
3,A,E,-0.248887
4,A,F,-0.605836
...,...,...,...
71,T,R,0.088978
72,T,S,0.254695
73,T,T,0.189795
74,T,V,0.528718


In [4]:
# Define the four bases and the missing amino acid Tyrosine (Y)
bases = ["A", "G", "C", "T"]
missing_aa = "Y"

# Create a dataframe for the missing entries with coef = 0
missing_rows = pd.DataFrame({
    "base": bases,
    "aa": [missing_aa] * len(bases),
    "coef": [0.0] * len(bases)  # Assign coefficient 0
})

# Append the missing rows to the dataframe
df_avg_coef = pd.concat([df_avg_coef, missing_rows], ignore_index=True)

In [6]:
# Replace values in the 'base' column according to the mapping
base_replacement = {'A': 'DA', 'G': 'DG', 'C': 'DC', 'T': 'DT'}
df_avg_coef["base"] = df_avg_coef["base"].replace(base_replacement)
df_avg_coef

,base,aa,coef
0,DA,A,-0.451686
1,DA,C,-0.021628
2,DA,D,-0.313047
3,DA,E,-0.248887
4,DA,F,-0.605836
...,...,...,...
75,DT,W,0.215001
76,DA,Y,0.000000
77,DG,Y,0.000000
78,DC,Y,0.000000


In [8]:
# Define the file path for the new CSV file
output_file_path = "C2H2_avg_coef.csv"

# Save the updated dataframe to a CSV file
df_avg_coef.to_csv(output_file_path, index=False)
# # Print the output file path
# print(f"CSV file saved at: {output_file_path}")

In [22]:
# Reload the provided CSV file containing the average coefficients
file_path = "C2H2_avg_coef.csv"
df_avg_coef = pd.read_csv(file_path)

# Re-define the residue type mappings separately for amino acids and nucleotides

# Mapping for amino acids (standard 20 + extra)
aa_type_map = {
    'A': 0, 'C': 4, 'D': 3, 'E': 6, 'F': 13, 'G': 7, 'H': 8, 'I': 9, 'K': 11,
    'L': 10, 'M': 12, 'N': 2, 'P': 14, 'Q': 5, 'R': 1, 'S': 15, 'T': 16, 
    'V': 19, 'W': 17, 'Y': 18
}

# Mapping for nucleotides (DNA bases)
base_type_map = {
    'DA': 20, 'DG': 21, 'DC': 22, 'DT': 23
}

# Merge both dictionaries for a complete mapping
res_type_map = {**aa_type_map, **base_type_map}

# Define the size of the matrix (24x24)
matrix_size = 24

# Initialize the matrix with NaN values
import numpy as np
matrix = np.full((matrix_size, matrix_size), np.nan)

# Set diagonal elements to 0 (self-interaction is zero)
np.fill_diagonal(matrix, 0)

# Fill the matrix based on df_avg_coef
for _, row in df_avg_coef.iterrows():
    base, aa, coef = row["base"], row["aa"], row["coef"]
    
    # Ensure both base and aa exist in the mapping
    if base in res_type_map and aa in res_type_map:
        i, j = res_type_map[base], res_type_map[aa]
        
        # Fill both matrix[i, j] and matrix[j, i] to ensure symmetry
        matrix[i, j] = coef
        matrix[j, i] = coef

# Fill any remaining NaN values with 0
matrix = np.nan_to_num(matrix)

# Convert the matrix into a DataFrame for visualization
df_matrix = pd.DataFrame(matrix, index=range(matrix_size), columns=range(matrix_size))

# # Define the file path for saving the matrix
# output_matrix_path = "generated_matrix.csv"

# # Save the matrix to a CSV file
# df_matrix.to_csv(output_matrix_path, index=False)

gamma = []
for i in range(matrix_size):
    for j in range(i, matrix_size):
        gamma.append(-matrix[i, j])  # the lower the better


#print(len(gamma))

# Convert the gamma list into a single-column DataFrame
df_gamma = pd.DataFrame(gamma)

# Define the file path for saving the gamma values
output_gamma_path = "C2H2_gamma"

# Save the gamma values to a CSV file, ensuring one value per row
df_gamma.to_csv(output_gamma_path, index=False, header=False)

In [26]:
# Import necessary libraries
import pandas as pd

# Load the CSV file
file_path = "homeodomain.csv"
df = pd.read_csv(file_path)

# Drop rows where 'base' or 'aa' contain NA values
df_cleaned = df.dropna(subset=["base", "aa"])

# Compute the mean coefficient for each (base, aa) pair
df_avg_coef = df_cleaned.groupby(["base", "aa"])["coef"].mean().reset_index()

df_avg_coef


,base,aa,coef
0,A,A,-0.094219
1,A,C,0.077095
2,A,D,-0.633471
3,A,E,-0.086404
4,A,F,-0.170311
...,...,...,...
71,T,R,-0.052091
72,T,S,0.335717
73,T,T,0.131177
74,T,V,-0.008240


In [28]:
# Define the four bases and the missing amino acid Tyrosine (Y)
bases = ["A", "G", "C", "T"]
missing_aa = "Y"

# Create a dataframe for the missing entries with coef = 0
missing_rows = pd.DataFrame({
    "base": bases,
    "aa": [missing_aa] * len(bases),
    "coef": [0.0] * len(bases)  # Assign coefficient 0
})

# Append the missing rows to the dataframe
df_avg_coef = pd.concat([df_avg_coef, missing_rows], ignore_index=True)

In [30]:
# Replace values in the 'base' column according to the mapping
base_replacement = {'A': 'DA', 'G': 'DG', 'C': 'DC', 'T': 'DT'}
df_avg_coef["base"] = df_avg_coef["base"].replace(base_replacement)
df_avg_coef

,base,aa,coef
0,DA,A,-0.094219
1,DA,C,0.077095
2,DA,D,-0.633471
3,DA,E,-0.086404
4,DA,F,-0.170311
...,...,...,...
75,DT,W,0.011357
76,DA,Y,0.000000
77,DG,Y,0.000000
78,DC,Y,0.000000


In [32]:
# Define the file path for the new CSV file
output_file_path = "homeodomain_avg_coef.csv"

# Save the updated dataframe to a CSV file
df_avg_coef.to_csv(output_file_path, index=False)
# # Print the output file path
# print(f"CSV file saved at: {output_file_path}")

In [34]:
# Reload the provided CSV file containing the average coefficients
file_path = "homeodomain_avg_coef.csv"
df_avg_coef = pd.read_csv(file_path)

# Re-define the residue type mappings separately for amino acids and nucleotides

# Mapping for amino acids (standard 20 + extra)
aa_type_map = {
    'A': 0, 'C': 4, 'D': 3, 'E': 6, 'F': 13, 'G': 7, 'H': 8, 'I': 9, 'K': 11,
    'L': 10, 'M': 12, 'N': 2, 'P': 14, 'Q': 5, 'R': 1, 'S': 15, 'T': 16, 
    'V': 19, 'W': 17, 'Y': 18
}

# Mapping for nucleotides (DNA bases)
base_type_map = {
    'DA': 20, 'DG': 21, 'DC': 22, 'DT': 23
}

# Merge both dictionaries for a complete mapping
res_type_map = {**aa_type_map, **base_type_map}

# Define the size of the matrix (24x24)
matrix_size = 24

# Initialize the matrix with NaN values
import numpy as np
matrix = np.full((matrix_size, matrix_size), np.nan)

# Set diagonal elements to 0 (self-interaction is zero)
np.fill_diagonal(matrix, 0)

# Fill the matrix based on df_avg_coef
for _, row in df_avg_coef.iterrows():
    base, aa, coef = row["base"], row["aa"], row["coef"]
    
    # Ensure both base and aa exist in the mapping
    if base in res_type_map and aa in res_type_map:
        i, j = res_type_map[base], res_type_map[aa]
        
        # Fill both matrix[i, j] and matrix[j, i] to ensure symmetry
        matrix[i, j] = coef
        matrix[j, i] = coef

# Fill any remaining NaN values with 0
matrix = np.nan_to_num(matrix)

# Convert the matrix into a DataFrame for visualization
df_matrix = pd.DataFrame(matrix, index=range(matrix_size), columns=range(matrix_size))

# # Define the file path for saving the matrix
# output_matrix_path = "generated_matrix.csv"

# # Save the matrix to a CSV file
# df_matrix.to_csv(output_matrix_path, index=False)

gamma = []
for i in range(matrix_size):
    for j in range(i, matrix_size):
        gamma.append(-matrix[i, j])  # the lower the better


#print(len(gamma))

# Convert the gamma list into a single-column DataFrame
df_gamma = pd.DataFrame(gamma)

# Define the file path for saving the gamma values
output_gamma_path = "homeodomain_gamma"

# Save the gamma values to a CSV file, ensuring one value per row
df_gamma.to_csv(output_gamma_path, index=False, header=False)